## Fitting photoswitching data

The code in this notebook is that which was used to fit experimental photoswitch pH recovery data in Wimberger et al. 

The code incorporates simulation of the kinetic problem with solution of the equilibrium problem for reactions identified as fast equilibria. It proves able to fit experimental data reasonably well.

The code is commented throughout. The first two code cells involve set-up of necessary functions. The third cell showcases an example of a simulation of an arbitrary system, given well-defined kinetic and equilibrium parameters. The fourth and following cells demonstrate how the differential evolution algorithm can be used to adjust kinetic/thermodynamic/quantity parameters to fit several experimental data files simultaneously.

The code benefits from the numba package to dramatically increase the efficiency of several routines which are used heavily during the simulation.

The differential evolution algorithm is parallelised. On a medium-high end desktop, a typical fitting process (fitting 5 data sets) took up to 10 minutes with 8 cores. In that example, the code was run using Jupyter on Windows Subsystem for Linux.

In [ ]:
# Import necessary modules. 
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import scipy as sp
from numba import jit # this package is very valuable for increasing the efficiency of the script
#from optimparallel import minimize_parallel  # not required but can be useful if you want to do parallel LBGFS
from IPython.display import display, Markdown # lets us print tables of values tables using standard iPython 


In [ ]:
# Set everything up - all functions for fitting
# 
# #https://github.com/numba/numba/issues/1269#issuecomment-702665837
# This solution enables us to apply the overloaded np.prod function along a single axis, see link above
@jit(nopython=True)
def apply_along_axis_0(func1d, arr):
    """Like calling func1d(arr, axis=0)"""
    if arr.size == 0:
        raise RuntimeError("Must have arr.size > 0")
    ndim = arr.ndim
    if ndim == 0:
        raise RuntimeError("Must have ndim > 0")
    elif 1 == ndim:
        return func1d(arr)
    else:
        result_shape = arr.shape[1:]
        out = np.empty(result_shape, arr.dtype)
        _apply_along_axis_0(func1d, arr, out)
        return out

# This solution enables us to apply the overloaded np.prod function along a single axis, see link above
@jit
def _apply_along_axis_0(func1d, arr, out):
    """Like calling func1d(arr, axis=0, out=out). Require arr to be 2d or bigger."""
    ndim = arr.ndim
    if ndim < 2:
        raise RuntimeError("_apply_along_axis_0 requires 2d array or bigger")
    elif ndim == 2:  # 2-dimensional case
        for i in range(len(out)):
            out[i] = func1d(arr[:, i])
    else:  # higher dimensional case
        for i, out_slice in enumerate(out):
            _apply_along_axis_0(func1d, arr[:, i], out_slice)

# This function solves the equilibrium concentration problem based on equilibrium constants, adapted
# from Maeder and co workers: https://doi.org/10.1016/S0922-3487(07)80006-2
@jit(nopython=True)
def getConcs(initComponentConc,logKaSPH,logKaMCH, initguess):
    # First define equilibrium matrix:

    # this matrix defines the equilibrium expression for each
    # equilibrium reaction
    #                 SP MC H SPH MCH OH 
    eqMat = np.array([[1.0,0, 0, 1,  0,  0],  # sp
                      [0.0,1, 0, 0,  1,  0],     # mc
                      [0.0,0, 1, 1,  1, -1]])     # h


    # now we give a vector containing the (log) equilibrium constants. Be careful
    # with sign because these constants are for formation of the complexes
    # above from the "pure components". Most importantly this means a 
    # Ka (for acidicity) is defined as its inverse (or negative, logged)

    # Read the comment above, even though it is long, to understand the choice of signs for Kas here
    logK =  np.array([ 0,0,0,logKaSPH,logKaMCH,-14])
    K = 10.0**logK

    # make an initial guess. This doesn't need to be good - it should just
    # be nearly-equally bad for all parameters
    initguess = np.array([0,0,0]) + 1e-18

    # solve the equilibrium problem above using the Newton Raphson method
    comp,spec = DoNR(eqMat,K,initComponentConc,initguess)
    return spec

# This function implements the Newton Raphson method for solving the equilibrium problem
# following: https://doi.org/10.1016/S0922-3487(07)80006-2
@jit(nopython=True)
def DoNR(eqMat,K,initComponentConc,guessCompConc):
    nspecies = len(K)
    ncomp = len(initComponentConc)
    
    initComponentConc[initComponentConc<=0] = 1e-18 # avoids numerical errors. Changed to <=0 rather than == 0 because sometimes small negative errors appear which makes
                                                    # the optimisation impossible
    for iter in range(0,500):

        conc = guessCompConc

        specmat = conc.repeat(nspecies).reshape((-1, nspecies))

        eq3pt47 = apply_along_axis_0(np.prod,specmat**eqMat)
        speciesConc = K * eq3pt47  # eq 3.48

        compTotCalc = eqMat @ speciesConc
        deltaComp = initComponentConc - compTotCalc

        if np.all(np.abs(deltaComp) < 1e-15):
            return compTotCalc,speciesConc

        J = np.zeros((ncomp,ncomp))

        # calculate Jacobian
        for ii in range(0,ncomp):
            for jj in range(0,ncomp):
                J[ii,jj] = np.sum(eqMat[ii,:]*eqMat[jj,:]*speciesConc)
                J[jj,ii] = J[ii,jj]

        deltaConc = np.linalg.lstsq(J, deltaComp)[0].T * conc
        conc += deltaConc

        while np.any(conc <= 0):
            deltaConc = deltaConc/2
            conc -= deltaConc
            if np.all(np.abs(deltaConc)<1e-15):
                break

    # Only reached if the optimisation does not complete within the available steps (500)
    print("Failed to converge in equilibrium solver doNR")
    print(deltaComp)
    print(initComponentConc)


#solve differential equations = concentration of different species over time
# y is SP MC SPH MCH
@jit(nopython=True)
def rhs(y, t, kopen, kclose, kcis, ktrans, logKaSPH, logKaMCH, H, OH):
    # make array of total component concentrations:
    # SP, MC, H
    # where... SP = SP+SPH; MC = MC+MCH; H = H-OH  (since our H and OH here are from the 0th iteration)
    compConcs = np.array([y[0]+y[2],y[1]+y[3],H-OH+y[2]+y[3]]) ## 2021-10-19 -- add SPH and MCH to total H conc
    # yEq is: SP, MC, H, SPH, MCH, OH

    # note that logKa here is an association constant (in ML binding terms), not an acidity constant.
    yEq = getConcs(compConcs,logKaSPH,logKaMCH,np.array([y[0], y[1], H-OH]))
    
    # here Ro (etc) are the components of d[Y]/dt for different components and are returned
    # in the array res
    Ro = kopen * yEq[0] 
    Rc = kclose * yEq[1] 
    Rcis = kcis * yEq[3] 
    Rtrans = ktrans * yEq[4] 
     # SP, MC, H, SPH, MCH, OH

    # SP, MC, SPH, MCH
    # so d[SP]/dt is Rc-Ro = kclose*[MC]eq - kopen*[SP]eq etc
    res = np.array([Rc-Ro,Ro-Rc,Rtrans-Rcis,Rcis-Rtrans])
  
    return res


# This function handles the simulation setup, prepares the initial derived parameters (like [H])
# and runs the simulation
# the input is a dict of system parameters.
def simulate(params):
    pKalight = params['pKalight']
    pHlight = params['pHlight']
    Kalight = 10**(-pKalight)
    kopen = params['kopen']
    kclose = params['kclose']
    kcis = params['kcis']  # these are set to zero and not allowed to vary: i.e. this process does not happen
    ktrans = params['ktrans']  # these are set to zero and not allowed to vary: i.e. this process does not happen
    # kocis = params['kocis']
    # kccis = kocis*Kalight
    pKaMCH = params['pKaMCH']
    #logKaSPH = params['pKalight']
    totalconc = params['totalconc']
    bleach = params['bleach']  # an empirical per-sample parameter which is intended to capture a difference in bleach extent.
    timerange = params['timerange']
    dt = params['dt']

    #mole fractions at pHlight
    XcisMCH = 1/((10**(pHlight-pKalight)+1))
    XSP = 1 - XcisMCH

    #starting concentrations
    SPH = bleach*(XcisMCH * totalconc)  #the model calls cis-MCH "SPH" for simplicity
    SP = bleach*(XSP * totalconc)
    MC = 0 #assumed to be zero in all cases. i.e. for incomplete bleach (`bleach` > 0) we add the remainder as MCH. Note that this can introduce an error into the total [H] but is expected to be minor.

    H = 10**(-pHlight) #defined by pH
    MCH =  (1-bleach)*totalconc #assumed to be zero at a complete bleach, otherwise note. Note caution about potential error above.
    OH = 0 #defined by pH and Kw

    Htot = getHtot(params) # get total amount of H (i.e. sum of free H, SPH, and MCH - these are necessary for solving the equilibrium system)

    #define array for time
    tout = np.arange(0, timerange, dt)
    #define rate constants
    k_vals = kopen, kclose, kcis, ktrans, pKalight, pKaMCH

    #solve equilibrium problem for starting composition
    compConcs = np.array([SP+SPH,MC+MCH, Htot])  
    yEq = getConcs(compConcs,pKalight,pKaMCH,np.array([SP, MC,  H]))
    
    # generate start concentrations for kinetic model: SP, MC, SPH, MCH  (SPH here = cisMCH))
    y0 = [yEq[0],yEq[1],yEq[3],yEq[4]]
    #retrieve concentrations over time
    yout = odeint(rhs, y0, tout, args=(*k_vals,H,OH),rtol=1e-15)
       
    return tout,yout

# calculates total concentration of [H+]
def getHtot(params):
    pHlight = params['pHlight']
    pKalight = params['pKalight']
    totalconc = params['totalconc']
    bleach = params['bleach']

    #mole fractions at pHlight
    XcisMCH = 1/((10**(pHlight-pKalight)+1))
    XSP = 1 - XcisMCH

    #starting concentrations
    SPH = bleach*(XcisMCH * totalconc)
    SP = bleach*(XSP * totalconc)
    MC = 0 #assumed to be zero at a complete bleach

    H = 10**(-pHlight) #defined by pH
    MCH =  (1-bleach)*totalconc #assumed to be zero at a complete bleach
    Htot = H+SPH+MCH
    return Htot


def fitFun(inp,params,realData,inpList,indpH=False,indBleach=False):
    # we are fitting # pkalight , kclose , pKaMCH

    for ii,p in enumerate(inpList):
        params[p] = inp[ii]

    res = []

    for ii,dd in enumerate(realData):

        params['pHlight'] = dd[0,1]
        params['timerange'] = np.max(dd[:,0])

        # # applies a per-run pH offset
        if indpH is True:
            print("Individual pH offset not implemented, edit second cell (line following this comment) to restore.")
        #     params['pHlight'] = dd[0,1]-params['pHOffset'+str(ii)]

        # allows the bleach% to vary for every sample.
        if indBleach is True:
            params['bleach'] = params['bleach'+str(ii)]

 
        # simulate the system based only on the parameters
        tout,yout = simulate(params)  # yout just contains the kinetic parameters, i.e. SP, MC, SPH, MCH
        youtFull = []

        #calculate the full concentration profiles for all species by solving the equilibrium problem at each step.
        for ii,yy in enumerate(yout):
            compConcs = np.array([yy[0]+yy[2],yy[1]+yy[3], getHtot(params)])  # here we use getHtot(params) here - i.e. using the initial [H]_tot, because it is not in ytot (not a kinetic parameter)
            yEq = getConcs(compConcs,params['pKalight'],params['pKaMCH'],np.array([yy[0], yy[1],  10**(-params['pHlight'])]))
            youtFull.append(yEq)  

        # calculate residuals

        realTimes = dd[:,0]
        realYs = dd[:,1]
        
        concH = np.array(youtFull)[:,2]
        concH = np.interp(realTimes,tout,concH)
        pHout = -np.log10(concH)
        rangeExpt = np.max(realYs) - np.min(realYs)
        

        # Mean-squared error scaled
        # We scale the error by the total range in the experimental data, so that runs with small pH changes are equally-weighted to those with large changes
        # the factor of 1000 is just so the typical final value of the error is about 1-10 for a good fit.
        res.append(1000*np.mean( (pHout-realYs)**2/rangeExpt ))
        # mae scaled
        #res.append(1000*np.mean(np.abs((pHout - realYs)/rangeExpt)))
        # sse only
        #res.append(np.sum( (pHout-realYs)**2 ))
        # sse scaled
        #res.append(np.sum( (pHout-realYs)**2/rangeExpt  ))

    return np.mean(res)



# this is a callback function which can be passed to differential_evolution. It prints the values of the parameters at the step.
def cb(xk,convergence=None):
    print('Curr vals: {}'.format(xk))


# this function takes the optimised values of the parameters and calculates the resulting profiles
# it uses the starting pH (i.e the pH when the light is turned off) from the experimental data files.
# The plots show both experimental and calculated data (top) and the difference between them (bottom).
# Only the calculated pH data (not any other concentration) are plotted.
def showFits(inp,params,realData,inpList,indpH=False,indBleach=False,title='Fit'):
    # we are fitting # pkalight , kclose , pKaMCH

    for ii,p in enumerate(inpList):
        params[p] = inp[ii]

    res = []
    yall = []
    allpH = []
    for ii,dd in enumerate(realData):

        params['pHlight'] = dd[0,1]-params['pHOffset']
        params['timerange'] = np.max(dd[:,0])

        if indpH is True:
            params['pHlight'] = dd[0,1]-params['pHOffset'+str(ii)]

        if indBleach is True:
            params['bleach'] = params['bleach'+str(ii)]


        tout,yout = simulate(params)
        youtFull = []
        for ii,yy in enumerate(yout):
            compConcs = np.array([yy[0]+yy[2],yy[1]+yy[3], getHtot(params)])  
            yEq = getConcs(compConcs,params['pKalight'],params['pKaMCH'],np.array([yy[0], yy[1],  10**(-params['pHlight'])]))
            youtFull.append(yEq)  
        
        yall.append(youtFull)
        # calculate residuals

        realTimes = dd[:,0]
        realYs = dd[:,1]
        
        concH = np.array(youtFull)[:,2]
        concH = np.interp(realTimes,tout,concH)
        pHout = -np.log10(concH)
        allpH.append(pHout)
        res.append((pHout - realYs))

    

    nwide = len(realData)
    plt.subplots(2,nwide,figsize=(nwide*4,10))
    for ii,dd in enumerate(realData):
        plt.subplot(2,nwide,ii+1)
        plt.title(title+' start pH {}'.format(dd[0,1]))
        plt.plot(dd[:,0],dd[:,1],label='expt')
        plt.plot(dd[:,0],allpH[ii],label='fit')
        plt.legend()
        plt.ylabel('pH')
        plt.xlabel('Time (s)')
        plt.subplot(2,nwide,nwide+ii+1)
        plt.plot(dd[:,0],res[ii])
        plt.xlabel('Time (s)')
        plt.ylabel('Difference between calc expt pH')


# This function can be uncommented (and you will also need to uncomment the import minimize_parallel part)
# if you wish to optimised using LBFGS only, and not using differential evolution.
# The function takes advantage of minimize_parallel to massively speed up the process.
# It auto-calculates the initial values as the middle of the bounds, but this can be overridden by passing the argument x0.

# def doLBFGSMinim(args,bounds,options=None,parallel=None,x0=None,printTable=True):
#     if x0 is None:
#         x0 = [(x[1]-x[0])/2 for x in bounds]

#     if options is not None:
#         if parallel is not None:
#             res = minimize_parallel(fitFun,x0,args=args,bounds=bounds,options=options,parallel=parallel)
#         else:
#             res = minimize_parallel(fitFun,x0,args=args,bounds=bounds,options=options)
#     else:
#         res = minimize_parallel(fitFun,x0,args=args,bounds=bounds)#,options=options,parallel=parallel)

#     if printTable:
#         tStr = "| Value | Initial range | Opt  | Jac |\n"
#         tStr += "|---|---|---|---|\n"
#         for i,k in enumerate(bounds):
#             tStr += "| {}  |  {:.2e} – {:.2e} | {:.2e} | {:.2e} | \n".format(args[2][i],k[0],k[1],res.x[i],res.jac[i])

#         tStr += "\n\nTerminated with f(x) = {:.2e}; success status = {}".format(res.fun,res.success)
#         display(Markdown(tStr))

#     return res


def printParams(args,bounds,res):
    tStr = "| Value | Initial range | Opt  |\n"
    tStr += "|---|---|---|---|\n"
    for i,k in enumerate(bounds):
        tStr += "| {}  |  {:.2e} – {:.2e} | {:.2e} | \n".format(args[2][i],k[0],k[1],res.x[i])
    tStr += "\n\nTerminated with f(x) = {:.2e}; success status = {}".format(res.fun,res.success)
    display(Markdown(tStr))

# This function saves the fit results as CSV files.
# It reuses a lot of code from the showFits(...) function, and could be made more concise (TODO).
def saveFitResults(inp,params,realData,inpList,indpH=False,indBleach=False,title=None):
    if title is None:
        print("Please give a value to parameter filestem and try again. filestem is what should appear as the start of each output csv filename")
        print("Nothing has been saved!")
        return


    for ii,p in enumerate(inpList):
        params[p] = inp[ii]

    res = []
    yall = []
    allpH = []
    for ii,dd in enumerate(realData):

        params['pHlight'] = dd[0,1]-params['pHOffset']
        params['timerange'] = np.max(dd[:,0])

        if indpH is True:
            params['pHlight'] = dd[0,1]-params['pHOffset'+str(ii)]

        if indBleach is True:
            params['bleach'] = params['bleach'+str(ii)]


        tout,yout = simulate(params)
        youtFull = []
        for ij,yy in enumerate(yout):
            compConcs = np.array([yy[0]+yy[2],yy[1]+yy[3], getHtot(params)])  ### 2021-10-19: calc [H]tot correctly
            yEq = getConcs(compConcs,params['pKalight'],params['pKaMCH'],np.array([yy[0], yy[1],  10**(-params['pHlight'])]))
            youtFull.append(yEq)  
        
        yall.append(youtFull)
        # calculate residuals

        realTimes = dd[:,0]
        realYs = dd[:,1]
        
        concH = np.array(youtFull)[:,2]
        concH = np.interp(realTimes,tout,concH)
        pHout = -np.log10(concH)
        allpH.append(pHout)

        dataOut = np.vstack((realTimes,realYs,pHout))

        filename = "{}-run-{}-pHlight-{}-simulated.csv".format(title,ii,params['pHlight'])
        with open(filename, 'w') as ff:
            ff.write('Time (s), Real data (pH), Simulated data (pH)\n')
            np.savetxt(ff, dataOut.T, delimiter=",")



In [ ]:
## This cell shows an arbitrary example of a simulation of a system without reference to experimental data 

params = {'pKalight': 1.78,          #experimental value pKaSP
          'kopen': 0.00172725, #experimental value
          'kclose': 0.044214392, #experimental value
          'kcis': 0,  #likely to be very very slow
          'ktrans': 0,  #likely to be very very slow
          'kocis': 1e8, #arbitrarily defined as "fast protonation"
          'pKaMCH': 6.97,
            'pHlight': 3.16, #pH value under irradiation, to be entered
            'totalconc': 0.0121, #total concentration of SP,MCH,MC,cis-MCH in solution
            'timerange': 200, #for low pH values >1000 for pHlight values 2.5 and above 200 suffices
            'dt': 5, #decrease this when lines are not smooth          
            'bleach': 1

         }


# specify pHs to simulate 
pHList = [3.2]

# prep output lists
tout = []
res = []

for pH in pHList:
    params['pHlight'] = pH
    # simulate the differential equation and equilibrium problem
    tout,yout = simulate(params)
    youtFull = []
    # the simulate() function only returns the conentraitons of kinetic species. In this loop we calculate
    # the concentrations of equilibrium-controlled parameters
    for ii,yy in enumerate(yout):
        compConcs = np.array([yy[0]+yy[2],yy[1]+yy[3], getHtot(params)])  
        yEq = getConcs(compConcs,params['pKalight'],params['pKaMCH'],np.array([yy[0], yy[1],  10**(-pH)]))
        youtFull.append(yEq)
    res.append(np.array(youtFull))


for ii,rr in enumerate(res):
    plt.plot(tout,-np.log10(rr[:,2]),label="pHLight = {}".format(pHList[ii]))

plt.xlabel('Time (s)')
plt.ylabel('pH')
plt.legend()
plt.show()



In [ ]:
# This cell and following cells provide examples of how series of raw data can be used to
# fit specified paramters in a model using differential evolution. The data files are specified 
# as CSV files given by basepath and path variables. For each CSV file, there is no header, and 
# the first column is time (s) and the second is pH.

#  code cell A6. Fitting compound 3d, dilute, 100% light intensity [nb some temperature fluctuations exist here]

#kinetic parameters starting from 100% bleach
params27dil = {'pKalight': 1.78,          #experimental value pKaSP
          'kopen': 0.00172725, #experimental value
          'kclose': 0.044214392, #experimental value
          'kcis': 0,  #likely to be very very slow
          'ktrans': 0,  #likely to be very very slow
          'kocis': 1e8, #arbitrarily defined as "fast protonation"
          'pKaMCH': 6.97,
            'pHlight': 3.2, #pH value under irradiation, to be entered
            'totalconc': 0.0009, #total concentration of SP,MCH,MC,cis-MCH in solution
            'timerange': 400, #for low pH values >1000 for pHlight values 2.5 and above 200 suffices
            'dt': 5, #decrease this when lines are not smooth          
            'bleach': 1,
            'pHOffset': 0
         }

# specify paths
basepath='/home/user/'
paths=['new-wim27-dil-1-100pc.csv','new-wim27-dil-2-100pc.csv','new-wim27-dil-3-100pc.csv','new-wim27-dil-4-100pc.csv','new-wim27-dil-5-100pc.csv']

inData = []

for pp in paths:
    p = basepath+pp
    dd = np.genfromtxt(p,delimiter=',')
    inData.append(dd)


# specify the limits for each parameter to be optimised. The optimisation parameters are identified in 
# inpList below.
fitbounds = [(1.75,1.81),(0.01,0.053),(6.89,7.05),(1e-4,2e-3),(8e-4,10e-4),(0.7,1.0),(0.7,1.0),(0.7,1.0),(0.7,1.0),(0.7,1.0)]

inpList = ['pKalight','kclose','pKaMCH','kopen','totalconc','bleach0','bleach1','bleach2','bleach3','bleach4']

args=(params27dil,inData,inpList,False,True)
resDEA7=sp.optimize.differential_evolution(fitFun,fitbounds,args=args, maxiter=100,polish=True,workers=12)
showFits(resDEA7.x,*args,title="3d dilute 100% diffevol cell A7")
saveFitResults(resDEA7.x,*args,title="3d dilute 100% diffevol cell A7")
printParams(args,fitbounds,resDEA7)
print(resDEA7)  #note, the final value of fun (i.e. the normalized mean squared error) is not guaranteed to be the same as that in the accompanying paper, due to differences in the differential evolution 
# random seed when the code is re-run

In [ ]:
# B4: Compound 3d, dilute, 25% light.

basepath='/home/user/'
paths=['new-wim27-dil-1-25pc.csv','new-wim27-dil-2-25pc.csv','new-wim27-dil-3-25pc.csv','new-wim27-dil-4-25pc.csv','new-wim27-dil-5-25pc.csv']

inData = []

for pp in paths:
    p = basepath+pp
    dd = np.genfromtxt(p,delimiter=',')
    inData.append(dd)

params27dil = {'pKalight': 1.78,          #experimental value pKaSP
          'kopen': 0.00172725, #experimental value
          'kclose': 0.044214392, #experimental value
          'kcis': 0,  #likely to be very very slow
          'ktrans': 0,  #likely to be very very slow
          'kocis': 1e8, #arbitrarily defined as "fast protonation"
          'pKaMCH': 6.97,
            'pHlight': 3.2, #pH value under irradiation, to be entered
            'totalconc': 0.0009, #total concentration of SP,MCH,MC,cis-MCH in solution
            'timerange': 400, #for low pH values >1000 for pHlight values 2.5 and above 200 suffices
            'dt': 5, #decrease this when lines are not smooth          
            'bleach': 1,
            'pHOffset': 0
         }

fitbounds = [(1.75,1.81),(0.01,0.053),(6.89,7.05),(5e-4,5e-3),(8e-4,10e-4),(0.7,1.0),(0.7,1.0),(0.7,1.0),(0.7,1.0),(0.7,1.0)]#),(-.2,.2)]#,(0.5,1.0),(0.5,1.0),(0.5,1.0)]

inpList = ['pKalight','kclose','pKaMCH','kopen','totalconc','bleach0','bleach1','bleach2','bleach3','bleach4']#,'bleach1','bleach2','bleach3','bleach4']#,'pHOffset' ]#,'bleach0','bleach1','bleach2']
args=(params27dil,inData,inpList,False,True)
resDEB4=sp.optimize.differential_evolution(fitFun,fitbounds,args=args,maxiter=100,polish=True,workers=12)
print(resDEB4)
showFits(resDEB4.x,*args,title="3d dilute 25% diffevol cell B4")
saveFitResults(resDEB4.x,*args,title="3d dilute 25% diffevol cell B4")
printParams(args,fitbounds,resDEB4)

In [ ]:
# 
# compound 3d high conc 25%

#kinetic parameters starting from 100% bleach
params27conc = {'pKalight': 1.78,          #experimental value pKaSP
          'kopen': 0.00172725, #experimental value
          'kclose': 0.044214392, #experimental value
          'kcis': 0,  #likely to be very very slow
          'ktrans': 0,  #likely to be very very slow
          'kocis': 1e8, #arbitrarily defined as "fast protonation"
          'pKaMCH': 6.97,
            'pHlight': 3.2, #pH value under irradiation, to be entered
            'totalconc': 0.0085, #total concentration of SP,MCH,MC,cis-MCH in solution
            'timerange': 400, #for low pH values >1000 for pHlight values 2.5 and above 200 suffices
            'dt': 5, #decrease this when lines are not smooth          
            'bleach': 1,
            'pHOffset': 0
         }

basepath='/home/user/'
paths=['new-wim27-conc-1-25pc.csv','new-wim27-conc-2-25pc.csv','new-wim27-conc-3-25pc.csv','new-wim27-conc-4-25pc.csv','new-wim27-conc-5-25pc.csv']

inData = []

for pp in paths:
    p = basepath+pp
    dd = np.genfromtxt(p,delimiter=',')
    inData.append(dd)



fitbounds = [(1.75,1.81),(0.01,0.053),(6.89,7.05),(1e-4,2e-3),(0.0075,0.0095),(0.7,1.0),(0.7,1.0),(0.7,1.0),(0.7,1.0),(0.7,1.0)]

inpList = ['pKalight','kclose','pKaMCH','kopen','totalconc','bleach0','bleach1','bleach2','bleach3','bleach4']
args=(params27conc,inData,inpList,False,True)
resDEF4=sp.optimize.differential_evolution(fitFun,fitbounds,args=args,maxiter=100,polish=True,workers=12)
print(resDEF4)
showFits(resDEF4.x,*args,title='3d conc 25% diffevol cell F4')
saveFitResults(resDEF4.x,*args,title="3d conc 25% diffevol cell F4")
printParams(args,fitbounds,resDEF4)

### Begin section where we study compound 3c

This involves new params which I will call paramsc

In [ ]:
#cell E4 - 25% light, dilute, compound 3c. 

#kinetic parameters starting from 100% bleach
paramsc = {'pKalight': 1.98,          #experimental value pKaSP
          'kopen': 0.0032, #experimental value
          'kclose': 0.18, #experimental value
          'kcis': 0,  #likely to be very very slow
          'ktrans': 0,  #likely to be very very slow
          'kocis': 1e8, #arbitrarily defined as "fast protonation"
          'pKaMCH': 7.18,
            'pHlight': 3.2, #pH value under irradiation, to be entered
            'totalconc': 0.0008, #total concentration of SP,MCH,MC,cis-MCH in solution
            'timerange': 400, #for low pH values >1000 for pHlight values 2.5 and above 200 suffices
            'dt': 5, #decrease this when lines are not smooth          
            'bleach': 1,
            'pHOffset': 0
         }

basepath='/home/user/'
paths=['new-wim28-dil-2-25pc.csv','new-wim28-dil-3-25pc.csv','new-wim28-dil-4-25pc.csv','new-wim28-dil-5-25pc.csv','new-wim28-dil-6-25pc.csv']

inData = []

for pp in paths:
    p = basepath+pp
    dd = np.genfromtxt(p,delimiter=',')
    inData.append(dd)

fitbounds = [(1.97,1.99),(0.11,0.20),(7.08,7.28),(1e-3,10e-3),(0.00065,0.00095),(0.7,1.0),(0.7,1.0),(0.7,1.0),(0.7,1.0),(0.7,1.0)]
inpList = ['pKalight','kclose','pKaMCH','kopen','totalconc','bleach0','bleach1','bleach2','bleach3','bleach4']

args=(paramsc,inData,inpList,False,True)
resDEE4=sp.optimize.differential_evolution(fitFun,fitbounds,args=args,maxiter=100,disp=True,polish=True,workers=12)
print(resDEE4)
showFits(resDEE4.x,*args,title='3c 25% E4 DE')
saveFitResults(resDEE4.x,*args,title='3c dilute 25% diffevol cell E4')
printParams(args,fitbounds,resDEE4)